## 1. Import Pakages

In [29]:
import torch
import torch.nn as nn
import torch.utils.data as data
import unidecode
import string
import random
import re
import time, math

In [2]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

In [3]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [4]:
file = unidecode.unidecode(open('../data/shakespeare.txt').read())
file_len = len(file)

print('file_len =', file_len)

file_len = 1115394


In [52]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())
print('')
print(len(random_chunk()))

ne, my sovereign, with all speed.

KING HENRY VI:
My Lord of Somerset, what youth is that,
Of whom you seem to have so tender care?

SOMERSET:
My liege, it is young Henry, earl of Richmond.

KING HENRY

201


In [25]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('hello'))
print(char_tensor(random_chunk()))

tensor([17, 14, 21, 21, 24])
tensor([11, 14, 14, 23, 94, 28, 29, 30, 13, 34, 18, 23, 16, 94, 17, 24, 32, 94,
        44, 94, 22, 10, 34, 94, 12, 24, 22, 25, 10, 27, 14, 96, 55, 17, 18, 28,
        94, 25, 27, 18, 28, 24, 23, 94, 32, 17, 14, 27, 14, 94, 44, 94, 21, 18,
        31, 14, 94, 30, 23, 29, 24, 94, 29, 17, 14, 94, 32, 24, 27, 21, 13, 77,
        96, 36, 23, 13, 94, 15, 24, 27, 94, 11, 14, 12, 10, 30, 28, 14, 94, 29,
        17, 14, 94, 32, 24, 27, 21, 13, 94, 18, 28, 94, 25, 24, 25, 30, 21, 24,
        30, 28, 96, 36, 23, 13, 94, 17, 14, 27, 14, 94, 18, 28, 94, 23, 24, 29,
        94, 10, 94, 12, 27, 14, 10, 29, 30, 27, 14, 94, 11, 30, 29, 94, 22, 34,
        28, 14, 21, 15, 73, 96, 44, 94, 12, 10, 23, 23, 24, 29, 94, 13, 24, 94,
        18, 29, 78, 94, 34, 14, 29, 94, 44, 68, 21, 21, 94, 17, 10, 22, 22, 14,
        27, 94, 18, 29, 94, 24, 30, 29, 75, 96, 48, 34, 94, 11, 27, 10, 18, 23,
        94, 44, 68])


In [30]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

inp, target = random_training_set()
print(inp)
print(target)

tensor([10, 29, 94, 18, 28, 68, 29, 96, 34, 24, 30, 27, 94, 32, 24, 27, 28, 17,
        18, 25, 68, 28, 94, 25, 21, 14, 10, 28, 30, 27, 14, 94, 44, 94, 28, 17,
        10, 21, 21, 94, 13, 24, 94, 32, 18, 29, 17, 94, 29, 17, 18, 28, 94, 32,
        18, 12, 20, 14, 13, 94, 12, 10, 18, 29, 18, 15, 15, 82, 96, 96, 40, 54,
        38, 36, 47, 56, 54, 77, 96, 55, 27, 30, 21, 34, 73, 94, 24, 15, 15, 18,
        12, 14, 27, 73, 94, 11, 14, 12, 10, 30, 28, 14, 94, 17, 14, 94, 17, 10,
        29, 17, 94, 28, 24, 22, 14, 94, 24, 15, 15, 14, 23, 12, 14, 28, 94, 18,
        23, 94, 17, 18, 22, 96, 29, 17, 10, 29, 94, 29, 17, 24, 30, 94, 32, 24,
        30, 21, 13, 28, 29, 94, 13, 18, 28, 12, 24, 31, 14, 27, 94, 18, 15, 94,
        29, 17, 24, 30, 94, 12, 24, 30, 21, 13, 28, 29, 73, 94, 21, 14, 29, 94,
        17, 18, 22, 96, 12, 24, 23, 29, 18, 23, 30, 14, 94, 18, 23, 94, 17, 18,
        28, 94])
tensor([29, 94, 18, 28, 68, 29, 96, 34, 24, 30, 27, 94, 32, 24, 27, 28, 17, 18,
        25, 68, 28, 94,

In [34]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden
    

In [35]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers=2).to(device)

In [40]:

inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())

out, hidden = model(inp, hidden)
print(out.size())

tensor([36])
torch.Size([2, 1, 100])
torch.Size([1, 100])


In [41]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [42]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)
        output_dist = output.data.view(-1).div(0.8).exp()
        #_,top_i = torch.max(output_dist,dim=0)
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [71]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()
    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j]
        y_ = torch.LongTensor([y_.item()]).to(device)
        y, hidden = model(x,hidden)
        loss += criterion(y, y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 tensor(3.4037, grad_fn=<DivBackward0>) 

e ce s  -n et t walet i s 
 ce  nep s ~hc ,
 Yic.issewlri. cap}e iaR
w hH h tmRou ds hRO sf>e ugnWees
hihnEyvoMh.wn  Afou as ni * ddi ood osmhoHe :d.Et eaoec}th,fioe0ejsiR ;tool:hK 



 tensor(2.4203, grad_fn=<DivBackward0>) 

bewsre as thiss ben;Tu thed ar on yt ant haor winniile giat or igh the.

Non thee son at lore to and in Bou anksan:
Borly anm ther of and:
Yhe or mat beas meme thee eecend pent in nenou dame I baat is 



 tensor(2.2073, grad_fn=<DivBackward0>) 

bleall githnt fist ghare blost the, loth ginst'r hour the we heell bohs Irenou afe nia keld I mof fnere'lE no int-anes the the ther, ay prinstfand mearis sows wo a sinow and he no'ld the therno wh:
Tha



 tensor(2.4625, grad_fn=<DivBackward0>) 

bcis thing, thech,
Mave prenting wordourturfan, and you
me my of bly way,
Dest
And raiss meome preare that in'd gribs me swartall shood not wither,
BY JPAIN):
I cond theed younce, wo gactend, broust fo



 tensor(2.1306, grad_fn=<DivB

KeyboardInterrupt: 